# Predict LFC Files From Other LFC Files

In [ ]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from astropy.constants import c
from scipy import interpolate
from scipy.optimize import minimize, least_squares, curve_fit
from mpfit import mpfit

from tqdm import tqdm
import seaborn as sns

from waveCal import *

In [ ]:
# Gather files

# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
even = np.arange(num_lfc_files) % 2
evn_mask = even.astype(bool)
odd_mask = (1 - even).astype(bool)

In [ ]:
patch_dict = patchAndDenoise(ckpt_files[odd_mask], file_times=lfc_times[odd_mask],
    K=2, running_window=9, num_iters=45, return_iters=True,
    line_cutoff=0.5, file_cutoff=0.5, fast_pca=False, verbose=True)

In [ ]:
plt.figure()
plt.title('PCA Coefficient 0')
plt.xlabel('Time [mjd]')
plt.ylabel('Coefficient')
plt.plot(patch_dict['times'],patch_dict['ec'][:,0],'.-')

## Interpolate in Time

In [ ]:
odd_wfits = []
odd_diffs = []
odd_shift = []
for nfile in tqdm(range(len(ckpt_files[evn_mask]))):
    file_name = ckpt_files[evn_mask][nfile]
    try:
        x,m,w,e = readParams(file_name)
    except ValueError as err:
        continue
    
    w_fit = interp_coefs_and_predict(lfc_times[evn_mask][nfile],patch_dict,
                                     t_interp_deg=3, x_interp_deg=3,
                                     new_x=x, new_m=m)
    
    odd_wfits.append(w_fit)
    odd_diffs.append(w-w_fit)
    odd_shift.append((w-w_fit)/w*c.value)

In [ ]:
all_odd_shifts = np.concatenate(odd_shift)
good_mask = np.isfinite(all_odd_shifts)
innie_mask = abs(all_odd_shifts[good_mask]-np.mean(all_odd_shifts[good_mask])) < (5*np.std(all_odd_shifts[good_mask]))
plt.hist(all_odd_shifts[good_mask][innie_mask],50);

In [ ]:
np.mean(all_odd_shifts[good_mask])

In [ ]:
np.median(all_odd_shifts[good_mask])

In [ ]:
np.mean(all_odd_shifts[good_mask][innie_mask])

In [ ]:
np.median(all_odd_shifts[good_mask][innie_mask])

In [ ]:
np.std(all_odd_shifts[good_mask])

In [ ]:
plt.plot(patch_dict['times'],patch_dict['ec'][:,0])